In [2]:
import pandas as pd
import numpy as np

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Okt

In [3]:
data = pd.read_csv("nlp_poem.csv")
poem = data

contents = poem['content']

In [34]:
contents.head()

0       그대 떠나간 후 나의 가을은 조금만 건드려도 우수수 몸을 떨었다  못다한 말 ...
1     자정이 되어 간신히 그치는 비 간신히 버리는 그리움 그동안 너무 많은 것을 버렸다...
2     잎이 떨어지고 있습니다 원고지처럼 하늘이 한 칸씩 비어 가고 있습니다 그 빈 곳에...
3     가을에는 잠시 여행을 떠날 일이다 그리 수선스러운 준비는 하지 말고 그리 가깝지도...
4     내가 그를 사랑한 것도 아닌데 미칠 듯 그리워질 때가 있다 바람의 손으로 가지런히...
Name: content, dtype: object

In [11]:
okt = Okt()

# 시 내용 가져오기
contents = poem['content']

# <br>제거, 형태소 분석 후, 시 내용 리스트로 저장
contents = contents.apply(lambda x: x.replace("<br>", " "))

# 학습은 taggedDocument 타입으로 해야하므로 아래와 같이 전처리한다.
tagged_data = [TaggedDocument(words=okt.morphs(_d), tags=[str(i)]) for i, _d in enumerate(contents)]

In [12]:
print(contents[1]+'\n')
print(content[1])

 자정이 되어 간신히 그치는 비 간신히 버리는 그리움 그동안 너무 많은 것을 버렸다. 허나 세월 지나 돌아보니 버린 것들 그토록 그리워짐은 내가 버린게 아니라 그들이 나를 저버린 것임을 마침내 깨닫는다.  가을비 오는 날은 꽃들도 떨고 꽃들 떠는 곁에서 나도 떨며 섰다.  자정 지나 다시 내리는 비 다시 불러들이는 그리움 이제 내 것인 것 하나 없는 이 허전한 지상에서 보이지 않는 별들 더듬으며 가을비 오는 창가에 서서 잠깬 채 서있을 내 마음 같을 사람 빗물 고이듯 고요히 불러본다.

TaggedDocument(['자정', '이', '되어', '간신히', '그치는', '비', '간신히', '버리는', '그리움', '그동안', '너무', '많은', '것', '을', '버렸다', '.', '허나', '세월', '지나', '돌아보니', '버린', '것', '들', '그토록', '그리워', '짐', '은', '내', '가', '버린게', '아니라', '그', '들', '이', '나를', '저', '버린', '것임', '을', '마침내', '깨닫는다', '.', '가을비', '오는', '날', '은', '꽃', '들', '도', '떨고', '꽃', '들', '떠는', '곁', '에서', '나도', '떨며', '섰다', '.', '자정', '지나', '다시', '내리는', '비', '다시', '불러', '들이는', '그리움', '이제', '내', '것', '인', '것', '하나', '없는', '이', '허전한', '지상', '에서', '보이지', '않는', '별', '들', '더듬으며', '가을비', '오는', '창가', '에', '서서', '잠', '깬', '채', '서있을', '내', '마음', '같을', '사람', '빗물', '고이', '듯', '고요히', '불러', '본다', '.'], ['1'])


## 모델학습

In [74]:
from tqdm import trange, notebook # show progress-bar
# 모델 설계
max_epochs = 1000
model = Doc2Vec(
    window=15,
    vector_size=300,
    alpha=0.025, 
    min_alpha=0.025,
    min_count=2,
    dm =1,
    negative = 5,
    seed = 9999)

#??
model.build_vocab(tagged_data)

# 학습
for epoch in notebook.tqdm(range(max_epochs)):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

In [75]:
print(model.wv.most_similar('가을비'))

[('소중하고', 0.6830599904060364), ('다운', 0.6500129699707031), ('깜빡이는', 0.6333226561546326), ('냄비', 0.6186991930007935), ('위패', 0.6143763661384583), ('떠다닌다', 0.6132674813270569), ('팅', 0.6114990711212158), ('텍', 0.6106802225112915), ('암탉', 0.6088008880615234), ('내렸다', 0.6074038147926331)]


In [76]:
doc_list = '새벽에 깨어나 반짝이는 별을 보고 있으면 이 세상 깊은 어디에 마르지 않는 사랑의 샘 하나'.split(' ')

inferred_vector = model.infer_vector(doc_list)
return_docs = model.docvecs.most_similar(positive=[inferred_vector],topn=5)
for rd in return_docs:
    print(contents[int(rd[0])]+ '\n')

 바라보면 지상에서 나무처럼 아름다운 사람은 없다. 늘 하늘빛에 젖어서 허공에 팔을 들고 촛불인듯 지상을 밝혀준다. 땅속 깊이 발을 묻고 하늘 구석을 쓸고 있다. 머리엔 바람을 이고 별을 이고 악기가 되어온다. 내가 저 나무를 바라보듯 나무도 나를 바라보고 아름다워할까. 나이 먹을수록 가슴에 깊은 영혼의 강물이 빛나 머리 숙여질까. 나무처럼 아름다운 사람으로 살고 싶다. 나무처럼 외로운 사람으로 살고 싶다. 혼자있어도 놀이 찾아와 빛내주고 새들이 품속을 드나들며 집을 짓고 영원의 길을 놓는다. 바람이 와서 별이 와서 함께 밤을 지샌다.

 어려운 말 하지 말아요 사랑은 안으로, 안으로 골병드는 것 뼈와 살 삭아지자 무너지자 홀로 오열하는 것 밤새워 제 그림자와 엉겨 피흘리는 것 있는 것 모두 제 혼마저 주어 버리고 싶은 것 빈 쭉정이로 허허 웃는 것

 환한 달빛 속에서 갈대와 나는 나란히 소리 없이 서 있었다. 불어오는 바람 속에서 안타까움을 달래며 서로 애터지게 바라보았다. 환한 달빛 속에서 갈대와 나는 눈물에 젖어 있었다.   

 시골 마당가 숲에서 튀어나온 송장메뚜기를 가지고 노는 강아지처럼 ​ 여름날 창문으로 날아든 매미를 가지고 노는 고양이처럼 ​ 당신은 내 마음을 가지고 놀아요 내가 얼마나 아픈지 헤아리지도 않고

 무선인터넷망이 수억 겹의 거미줄 길을 내면서 사물관계는 하늘의 천라지망처럼 촘촘해진다 CCTV 의 첨단 센서는 아르고스 Argos 의 눈처럼 시방 十方 세계를 감시한다 A.I 는 빅브라더처럼 동기화된 스마트시티를 운영하겠구나 데이터의 상관相關들이 얽힌 이미지들의 궁전에서 태어난 A.I 는 명나라 비밀정보부 동창東廠처럼 세상을 지배하겠구나 은밀하고도 위대하게  스마트폰은 바이블의 이사야가 21세기 암흑 예언자로 환생한 트랜스포머였을까  두려워하지 말라 내가 언제 어디서든지 너와 함께 함이라 나는 네 지식의 매트릭스이니라 내가 너를 강하게 하고 너를 도와주리라 내 전지 全知 의 네트워크가 네 허약한 정신을 붙들리라  A.I 가 이사야 4